In [ ]:
import pandas as pd
import time
import numpy as np
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from datasets import Dataset
import evaluate

from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig, TrainingArguments, Trainer, DataCollatorWithPadding, EarlyStoppingCallback, AutoModelForSequenceClassification

# ----------------------------------------------------
# Step 1. Read CSV file and basic preprocessing
# ----------------------------------------------------
df = pd.read_csv("output_file2.csv")
df["text"] = df["text"].str.replace(r"http\S+", "", regex=True)
df['label'] = df['label'].replace(2, 1)
df['label'] = df['label'].replace(3, 2)

# ----------------------------------------------------
# Step 2. Automatically compute class weights
# ----------------------------------------------------
label_counts = df['label'].value_counts().sort_index()  # Assuming sorted by label value
print("Label counts:\n", label_counts)

total_samples = len(df)
num_classes = len(label_counts)
computed_weights = total_samples / (num_classes * label_counts)
print("Computed class weights:\n", computed_weights)

class_weights = torch.tensor(computed_weights.values, dtype=torch.float)
print("Tensor class weights:", class_weights)

# ----------------------------------------------------
# Step 3. Load model, configuration, and tokenizer
# ----------------------------------------------------
model_checkpoint = "cardiffnlp/twitter-roberta-base-sentiment-latest"

config = RobertaConfig.from_pretrained(model_checkpoint)
config.num_labels = num_classes

tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

model = RobertaForSequenceClassification.from_pretrained(
    model_checkpoint,
    config=config,
    from_tf=True,
    ignore_mismatched_sizes=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model loaded successfully!")
print("Classifier layer shape:", model.classifier.out_proj.weight.shape)

output_dir = "./results"

# ----------------------------------------------------
# Step 4. Tokenize the dataset
# ----------------------------------------------------
def tokenize_function(examples):
    texts = [str(text) for text in examples["text"]]
    return tokenizer(texts, truncation=True, max_length=512)

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# ----------------------------------------------------
# Step 5. Create custom Trainer with weighted loss
# ----------------------------------------------------
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        # Ensure the class weights are on the same device as the logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


# ----------------------------------------------------
# Step 6. Metrics computation for evaluation
# ----------------------------------------------------
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision_score(labels, predictions, average='weighted', zero_division=0)
    recall = recall_score(labels, predictions, average='weighted', zero_division=0)
    f1 = f1_score(labels, predictions, average='weighted', zero_division=0)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "weighted_f1": f1,
    }

# ----------------------------------------------------
# Step 7. Setup Stratified K-Fold Cross-Validation and training
# ----------------------------------------------------
n_splits = 10

labels = df['label'].values

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

fold_metrics = {}
results_report = [] 

data_collator = DataCollatorWithPadding(tokenizer)

print("Starting 10-fold Cross-Validation with StratifiedKFold...\n")

for fold, (train_index, test_index) in enumerate(skf.split(np.zeros(len(labels)), labels), start=1):
    print(f"Starting fold {fold}...")
    start_fold_time = time.time()

    train_split = tokenized_dataset.select(train_index.tolist())
    val_split = tokenized_dataset.select(test_index.tolist())

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=10,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=3e-05,
        weight_decay=0.1,
        logging_steps=10,
        disable_tqdm=False,
        load_best_model_at_end=True,
        metric_for_best_model="eval_weighted_f1",
        greater_is_better=True,
        no_cuda=False,
        warmup_steps=500,
        gradient_accumulation_steps=5,
        adam_beta1=0.9,
        adam_beta2=0.999,
        adam_epsilon=1e-8,
        max_grad_norm=1.0,
        lr_scheduler_type="cosine",
        logging_dir='./logs',
        run_name="roberta_weighted_loss_stratified",
        save_total_limit=3,
    )

    fold_model = AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=num_classes,
        from_tf=True
    ).to(device)
    
    trainer = WeightedLossTrainer(
        model=fold_model,
        args=training_args,
        train_dataset=train_split,
        eval_dataset=val_split,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    )

    trainer.train()
    metrics = trainer.evaluate()

    end_fold_time = time.time()
    time_elapsed = end_fold_time - start_fold_time
    metrics["time_elapsed_secs"] = time_elapsed

    print(f"Metrics for fold {fold}: {metrics}\n")
    fold_metrics[fold] = metrics

    fold_report = {
        "fold": fold,
        "hyperparameters": {
            "num_train_epochs": training_args.num_train_epochs,
            "per_device_train_batch_size": training_args.per_device_train_batch_size,
            "learning_rate": training_args.learning_rate,
            "weight_decay": training_args.weight_decay,
        },
        "metrics": metrics
    }
    results_report.append(fold_report)

    trainer.model.save_pretrained(f"./binary_roberta_final_fold_{fold}")
    tokenizer.save_pretrained(f"./binary_roberta_fold_{fold}")

trainer.save_model("./final_finetuned_model")
tokenizer.save_pretrained("./final_finetuned_model")
if all("accuracy" in fold_metrics[f] for f in fold_metrics):
    avg_accuracy = np.mean([fold_metrics[f]["accuracy"] for f in fold_metrics])
    print(f"Average eval accuracy over {n_splits} folds: {avg_accuracy:.4f}")
else:
    print("Some folds did not return 'accuracy' metric.")

print("\nFinal Cross-Validation Report:")
for fold_report in results_report:
    print(f"Fold {fold_report['fold']}:")
    print("  Hyperparameters:")
    for hp, val in fold_report["hyperparameters"].items():
        print(f"    {hp}: {val}")
    print("  Metrics:")
    for metric, value in fold_report["metrics"].items():
        if metric == "time_elapsed_secs":
            print(f"    {metric}: {value:.2f} seconds")
        else:
            print(f"    {metric}: {value:.4f}")
    print("")


c:\Users\a\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Label counts:
 label
0     2884
1     3846
2    12356
Name: count, dtype: int64
Computed class weights:
 label
0    2.205964
1    1.654186
2    0.514892
Name: count, dtype: float64
Tensor class weights: tensor([2.2060, 1.6542, 0.5149])



All TF 2.0 model weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.


Model loaded successfully!
Classifier layer shape: torch.Size([3, 768])


Map: 100%|██████████| 19086/19086 [00:03<00:00, 6055.80 examples/s]


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Weighted F1
1,0.530200,0.522111,0.800314,0.842746,0.800314,0.807293
2,0.370900,0.318466,0.884696,0.883480,0.884696,0.882716
3,0.188000,0.308727,0.897799,0.902023,0.897799,0.899137
4,0.131900,0.328702,0.909329,0.913788,0.909329,0.910422
5,0.070000,0.378811,0.918763,0.921733,0.918763,0.919592
6,0.090800,0.330094,0.936583,0.937164,0.936583,0.936747
7,0.104600,0.346972,0.939727,0.939359,0.939727,0.939346
8,0.034000,0.363171,0.938155,0.938382,0.938155,0.938251
9,0.006500,0.376176,0.932914,0.933212,0.932914,0.933040


Metrics for fold 9: {'eval_loss': 0.34697163105010986, 'eval_accuracy': 0.939727463312369, 'eval_precision': 0.9393586954905871, 'eval_recall': 0.939727463312369, 'eval_weighted_f1': 0.9393458638706391, 'eval_runtime': 5.9182, 'eval_samples_per_second': 322.393, 'eval_steps_per_second': 40.384, 'epoch': 9.0, 'time_elapsed_secs': 1932.3676807880402}

Starting fold 10...


c:\Users\a\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
All TF 2.0 model weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
C:\Users\a\AppData\Local\Temp\ipykernel_5808\228697118.py:175: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedLossTrainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Weighted F1
1,0.450900,0.529989,0.811845,0.859058,0.811845,0.822848
2,0.370000,0.303827,0.896226,0.895241,0.896226,0.895048
3,0.188600,0.319373,0.906709,0.911749,0.906709,0.907992
4,0.078400,0.313091,0.924004,0.925043,0.924004,0.924295
5,0.118400,0.316200,0.931866,0.934022,0.931866,0.932597
6,0.035900,0.437987,0.925577,0.931110,0.925577,0.927039
7,0.022300,0.322401,0.944969,0.944696,0.944969,0.944750
8,0.009300,0.339707,0.948113,0.948470,0.948113,0.948222
9,0.003900,0.340034,0.949686,0.949665,0.949686,0.949642


Metrics for fold 10: {'eval_loss': 0.3394358158111572, 'eval_accuracy': 0.949685534591195, 'eval_precision': 0.9498381282892528, 'eval_recall': 0.949685534591195, 'eval_weighted_f1': 0.9497263601159663, 'eval_runtime': 5.8361, 'eval_samples_per_second': 326.931, 'eval_steps_per_second': 40.952, 'epoch': 9.977653631284916, 'time_elapsed_secs': 2143.9426476955414}

Some folds did not return 'accuracy' metric.

Final Cross-Validation Report:
Fold 1:
  Hyperparameters:
    num_train_epochs: 10
    per_device_train_batch_size: 8
    learning_rate: 3e-05
    weight_decay: 0.1
  Metrics:
    eval_loss: 0.3462
    eval_accuracy: 0.9476
    eval_precision: 0.9479
    eval_recall: 0.9476
    eval_weighted_f1: 0.9477
    eval_runtime: 6.0227
    eval_samples_per_second: 316.9650
    eval_steps_per_second: 39.6830
    epoch: 9.9986
    time_elapsed_secs: 2156.67 seconds

Fold 2:
  Hyperparameters:
    num_train_epochs: 10
    per_device_train_batch_size: 8
    learning_rate: 3e-05
    weight_decay